<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/testing_set_covering_algorithms_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project requires Python 3.7 or above:



In [ ]:
import sys

assert sys.version_info >= (3, 7)

In [ ]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 55.8 MB/s eta 0:00:00


In [ ]:
import timeit
import heapq
import pulp


In [ ]:
#this generates an examples set where not all cameras might cover the stadiums_needed
import random

# Number of stadium areas
num_stadiums = 100

# Create the set of all unique stadium areas
stadiums_needed = set(range(1, num_stadiums + 1))

# Number of cameras
num_cameras = 30

# Define a function to generate random camera coverage
def generate_random_camera_coverage(stadiums, min_coverage=5, max_coverage=15):
    """Generate a random set of stadium areas covered by a camera."""
    stadiums_list = list(stadiums)  # Convert the set to a list
    coverage_size = random.randint(min_coverage, max_coverage)
    return set(random.sample(stadiums_list, coverage_size))  # Sample from the list

# Create a dictionary of camera locations and their randomly assigned coverage areas
camera_locations = {}
for i in range(1, num_cameras + 1):
    camera_name = f"camera{i}"
    camera_locations[camera_name] = generate_random_camera_coverage(stadiums_needed)

# Print the generated test case
print("stadiums_needed =", stadiums_needed)
print("camera_locations = {")
for camera, coverage in camera_locations.items():
    print(f'    "{camera}": {coverage},')
print("}")


stadiums_needed = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}
camera_locations = {
    "camera1": {96, 81, 20, 56, 93},
    "camera2": {97, 3, 6, 73, 16, 48, 51, 26},
    "camera3": {67, 4, 72, 42, 12, 45, 18, 51, 50, 21, 86, 53, 27, 93},
    "camera4": {100, 5, 37, 41, 83, 54, 90, 60},
    "camera5": {3, 8, 83, 20, 84, 55, 24},
    "camera6": {97, 99, 100, 10, 45, 14, 51, 58, 27, 61},
    "camera7": {4, 44, 82, 51, 50, 22, 94},
    "camera8": {98, 35, 67, 40, 55, 25, 59},
    "camera9": {96, 34, 99, 69, 43, 45, 77, 16, 20, 54, 22},
    "camera10": {64, 67, 99, 39, 16, 60},
    "camera11": {64, 96, 1, 7, 40, 11, 23, 61, 63},
    "camera12": {6

In [ ]:
#this generates an examples set where all cameras cover the stadiums_needed
import random

# Number of stadium areas
num_stadiums = 100

# Create the set of all unique stadium areas
stadiums_needed = set(range(1, num_stadiums + 1))

# Number of cameras
num_cameras = 30

# Define a function to generate random camera coverage
def generate_random_camera_coverage(stadiums, min_coverage=5, max_coverage=15):
    """Generate a random set of stadium areas covered by a camera."""
    stadiums_list = list(stadiums)  # Convert the set to a list
    coverage_size = random.randint(min_coverage, max_coverage)
    return set(random.sample(stadiums_list, coverage_size))  # Sample from the list

# Create a dictionary of camera locations and their randomly assigned coverage areas
camera_locations = {}
for i in range(1, num_cameras + 1):
    camera_name = f"camera{i}"
    camera_locations[camera_name] = generate_random_camera_coverage(stadiums_needed)

# Check which stadiums are not covered by any camera
covered_stadiums = set()
for coverage in camera_locations.values():
    covered_stadiums.update(coverage)

uncovered_stadiums = stadiums_needed - covered_stadiums

# Assign uncovered stadiums to random cameras until all stadiums are covered
if uncovered_stadiums:
    camera_names = list(camera_locations.keys())
    for stadium in uncovered_stadiums:
        random_camera = random.choice(camera_names)
        camera_locations[random_camera].add(stadium)

# Print the generated test case
print("stadiums_needed =", stadiums_needed)
print("camera_locations = {")
for camera, coverage in camera_locations.items():
    print(f'    "{camera}": {coverage},')
print("}")


stadiums_needed = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}
camera_locations = {
    "camera1": {69, 39, 7, 71, 42, 44, 45, 13, 82, 20, 84, 53, 24, 56},
    "camera2": {2, 3, 78, 16, 51, 22, 58},
    "camera3": {4, 5, 71, 41, 83, 89, 28, 30},
    "camera4": {32, 67, 68, 7, 14, 15, 49, 55, 88, 89, 30},
    "camera5": {64, 1, 97, 3, 98, 11, 46, 14, 81, 51, 52, 53, 87, 23, 93, 30},
    "camera6": {100, 16, 51, 84, 60, 62},
    "camera7": {99, 100, 35, 8, 73, 10, 77, 80, 28},
    "camera8": {40, 91, 82, 84, 55, 59},
    "camera9": {1, 66, 41, 73, 11, 45, 13, 48, 86, 89, 27, 29, 63},
    "camera10": {32, 97, 65, 70, 14, 56, 59, 31},
    "camera11

In [ ]:
stadiums_needed = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}
camera_locations = {
    "camera1": {66, 69, 71, 15, 80},
    "camera2": {4, 72, 75, 11, 47, 17, 52, 21, 86, 22, 88, 58, 28},
    "camera3": {34, 68, 73, 41, 77, 22, 58},
    "camera4": {96, 99, 36, 44, 52, 24, 25, 91, 60, 62},
    "camera5": {69, 42, 46, 80, 52, 23, 62, 95},
    "camera6": {32, 4, 8, 53, 54, 24, 61},
    "camera7": {36, 100, 7, 44, 77, 48, 80, 26, 92, 61, 30},
    "camera8": {34, 35, 70, 8, 79, 16, 52, 53, 24, 89, 59, 30},
    "camera9": {38, 70, 59, 62, 63},
    "camera10": {64, 33, 98, 35, 2, 5, 74, 44, 76, 49, 51, 21, 92, 31},
    "camera11": {97, 3, 68, 6, 10, 12, 93},
    "camera12": {40, 73, 47, 81, 83, 87, 60, 30, 95},
    "camera13": {5, 7, 9, 11, 93, 22, 23, 60, 29, 62},
    "camera14": {1, 67, 100, 38, 70, 72, 9, 15, 20, 92},
    "camera15": {70, 38, 9, 10, 75, 44, 51, 53, 89, 61},
    "camera16": {10, 42, 77, 46, 82, 51, 25, 29},
    "camera17": {64, 65, 100, 5, 68, 58, 40, 9, 11, 83, 25, 90, 94, 95},
    "camera18": {36, 100, 70, 39, 8, 6, 75, 83, 62, 30},
    "camera19": {33, 34, 45, 88, 89},
    "camera20": {98, 100, 7, 74, 81, 89, 91, 25, 63},
    "camera21": {32, 37, 43, 44, 52, 25, 58, 28},
    "camera22": {99, 5, 71, 11, 91},
    "camera23": {96, 2, 70, 73, 42, 11, 15, 16, 55, 25, 59, 60, 57},
    "camera24": {68, 9, 73, 10, 81, 53, 21, 24, 90, 88, 60},
    "camera25": {2, 36, 70, 39, 9, 46, 47, 16, 90, 91},
    "camera26": {33, 2, 90, 6, 8, 43, 47, 17, 58, 91, 92},
    "camera27": {99, 39, 27, 8, 7, 72, 60, 79, 80, 50, 21, 91, 92, 62},
    "camera28": {65, 1, 4, 9, 44, 77, 78, 46, 18, 19, 84, 86, 56, 25},
    "camera29": {32, 98, 27, 9, 14, 79, 81, 18, 19, 50, 51, 91, 92},
    "camera30": {34, 100, 10, 48, 81, 82, 18, 23, 61, 94, 63},
}

#this handles all cases like all the cameras not fully covering the the stadiums_needed
# this is the latest version for more explanation check out https://medium.com/p/a8e8b5bb6be0
def improved_set_covering(stadiums_needed, camera_locations):
    # Dictionary to store how many cameras cover each stadium and which cameras
    Count = {}
    final_camera_locations = set()
    max_count = 1  # Track the highest number of location covered by different cameras

    # Count occurrences of each stadium in the camera_locations
    for camera, stadiums in camera_locations.items():
        for stadium in stadiums:
            if stadium in Count:
                Count[stadium][0] += 1
                Count[stadium][1].add(camera)

                max_count = max(Count[stadium][0],max_count)
            else:
                Count[stadium] = [1, set([camera])]  # Initialize with count 1 and set of cameras

    # Function to calculate the number of stadiums in common with stadiums_needed
    def stadiums_in_common(camera):
        return len(stadiums_needed & camera_locations[camera])

    # Function to add the selected camera to the final list and update stadiums_needed
    def add_to_final(camera, stadiums_needed):
        if camera in final_camera_locations:
            return
        final_camera_locations.add(camera)
        stadiums_needed -= camera_locations[camera]

    i = 1  # This keeps track of the Count values
    min_set =set()
    while stadiums_needed and i <= max_count:
        for stadium, value in Count.items():
            if value[0] == i:
                min_set.update(value[1])  # Add all cameras covering the stadium with count i

        min_set-=final_camera_locations

        while min_set and stadiums_needed:
            if i >1:
              best_camera = max(min_set, key=stadiums_in_common)

              if stadiums_in_common(best_camera) == 0:
              # If the camera does not cover any needed stadiums, clear the set and break
                min_set.clear()
                break
              else:
                min_set.discard(best_camera)
                add_to_final(best_camera, stadiums_needed)

            elif i == 1:  # we don't have to go down the greedy approach , since all camera location that cover a stadium no other camera covers need to be included
              add_to_final(min_set.pop(), stadiums_needed)

        if not stadiums_needed:
            return final_camera_locations

        i+= 1  # Increase i if no cameras with i coverage

    return final_camera_locations,stadiums_needed

# Greedy Algorithm
def greedy_set_covering(stadiums_needed, camera_locations):
    final_locations =[]
    while stadiums_needed:
        best_location = None
        area_covered = set()
        for location, stadium_area in camera_locations.items():
            covered = stadiums_needed & stadium_area
            if len(covered) > len(area_covered):
                best_location = location
                area_covered = covered
        if best_location:
            final_locations.append(best_location)
            stadiums_needed -= area_covered
            del camera_locations[best_location]
        else:
            break
    return final_locations

# ILP Algorithm
def ilp_set_covering(stadiums_needed, camera_locations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    location_vars = pulp.LpVariable.dicts("Station", camera_locations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([location_vars[location] for location in camera_locations.keys()])
    for stadium in stadiums_needed:
        prob += pulp.lpSum([location_vars[location] for location in camera_locations.keys() if stadium in camera_locations[location]]) >= 1
    prob.solve()
    selected_locations = [locations for locations in camera_locations.keys() if location_vars[locations].varValue == 1]
    return selected_locations

# Heap-Based Algorithm
def heap_set_covering(stadiums_needed, camera_locations):
    final_locations =[]
    Count = {state: 0 for state in stadiums_needed}

    # Count occurrences of each area in the camera_locations
    for location, stadium_area in camera_locations.items():
        for area in stadium_area:
            if area in Count:
                Count[area] += 1

    heap = []

    # Function to calculate the number of stadium_area in common with stadium_needed
    def area_in_common(stadium_area):
        return len(stadiums_needed & stadium_area)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for location, stadium_area in camera_locations.items():
            if any(Count[state] == count for state in stadium_area):
                heapq.heappush(heap, (count, -area_in_common(stadium_area), location, stadium_area))

    # Process the heap
    while stadiums_needed and heap:
        _, _, location, stadium_area = heapq.heappop(heap)
        if location in final_locations:
            continue
        final_locations.append(location)
        stadiums_needed -= stadium_area
        for st in stadium_area:
            if st in Count:
                del Count[st]
        del camera_locations[location]

        # Rebuild the heap with remaining areas
        heap = []
        for count in range(1, max_count + 1):
            for location, stadium_area in camera_locations.items():
                if any(area in Count and Count[area] == count for area in stadium_area):
                    heapq.heappush(heap, (count, -area_in_common(stadium_area), location, stadium_area))

    return final_locations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
improved_result, improved_time = measure_runtime(improved_set_covering, stadiums_needed.copy(), camera_locations.copy())
greedy_result, greedy_time = measure_runtime(greedy_set_covering, stadiums_needed.copy(), camera_locations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, stadiums_needed.copy(), camera_locations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, stadiums_needed.copy(), camera_locations.copy())

# Print results
print(f"Improved Algorithm: length: {len(improved_result[0])} , {improved_result}, Time: {improved_time}")
print(f"Greedy Algorithm: length: {len(greedy_result)} ,{greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: length: {len(ilp_result)} ,{ilp_result}, Time: {ilp_time}")

# Summary
print(f"Improved Algorithm Time: {improved_time}")
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")


Improved Algorithm: length: 18 , ({'camera3', 'camera27', 'camera7', 'camera13', 'camera1', 'camera6', 'camera2', 'camera12', 'camera19', 'camera29', 'camera21', 'camera14', 'camera23', 'camera10', 'camera30', 'camera17', 'camera28', 'camera11'}, {13, 85}), Time: 0.00047971499952836893
Greedy Algorithm: length: 18 ,['camera10', 'camera28', 'camera27', 'camera23', 'camera30', 'camera2', 'camera11', 'camera12', 'camera6', 'camera1', 'camera14', 'camera7', 'camera19', 'camera21', 'camera3', 'camera13', 'camera17', 'camera29'], Time: 0.0003211350003766711
Heap-Based Algorithm: ['camera10', 'camera28', 'camera23', 'camera7', 'camera11', 'camera29', 'camera12', 'camera21', 'camera14', 'camera19', 'camera6', 'camera1', 'camera3', 'camera30', 'camera13', 'camera2', 'camera25', 'camera22'], Time: 0.005395161999331322
ILP Algorithm: length: 12 ,['camera1', 'camera2', 'camera3', 'camera11', 'camera14', 'camera16', 'camera17', 'camera19', 'camera20', 'camera23', 'camera25', 'camera28'], Time: 0.01

In [ ]:

# ILP result cameras
ilp_cameras = ['camera2', 'camera4', 'camera5', 'camera6', 'camera8', 'camera10',
               'camera12', 'camera13', 'camera15', 'camera22', 'camera26',
               'camera27', 'camera28', 'camera29']

# Combine the stadiums covered by the selected ILP cameras
covered_stadiums = set()
for camera in ilp_cameras:
    covered_stadiums.update(camera_locations[camera])

# Check if all stadiums are covered
all_stadiums_covered = stadiums_needed.issubset(covered_stadiums)

# Print the result
if all_stadiums_covered:
    print("All stadiums are covered by the selected ILP cameras.")
else:
    missing_stadiums = stadiums_needed - covered_stadiums
    print("Not all stadiums are covered. Missing stadiums:", missing_stadiums)


18
Not all stadiums are covered. Missing stadiums: {85, 13}


In [ ]:
states_needed = set(["mt", "wa", "or", "id", "nv", "ut", "ca", "az"])

stations = {
    "kone": set(["id", "nv", "ut"]),
    "ktwo": set(["wa", "id", "mt"]),
    "kthree": set(["or", "nv", "ca"]),
    "kfour":  set(["nv", "ut"]),
    "kfive": set(["ca", "az"])
}
# Greedy Algorithm
def greedy_set_covering(states_needed, stations):
    final_stations = set()
    while states_needed:
        best_station = None
        states_covered = set()
        for station, station_states in stations.items():
            covered = states_needed & station_states
            if len(covered) > len(states_covered):
                best_station = station
                states_covered = covered
        if best_station:
            final_stations.add(best_station)
            states_needed -= states_covered
            del stations[best_station]
        else:
            break
    return final_stations

# ILP Algorithm
def ilp_set_covering(states_needed, stations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    station_vars = pulp.LpVariable.dicts("Station", stations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([station_vars[station] for station in stations.keys()])
    for state in states_needed:
        prob += pulp.lpSum([station_vars[station] for station in stations.keys() if state in stations[station]]) >= 1
    prob.solve()
    selected_stations = [station for station in stations.keys() if station_vars[station].varValue == 1]
    return selected_stations

# Heap-Based Algorithm
def heap_set_covering(states_needed, stations):
    final_stations = set()
    Count = {state: 0 for state in states_needed}

    # Count occurrences of each state in the stations
    for station, station_states in stations.items():
        for state in station_states:
            if state in Count:
                Count[state] += 1

    heap = []

    # Function to calculate the number of states in common with states_needed
    def states_in_common(station_states):
        return len(states_needed & station_states)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for station, station_states in stations.items():
            if any(Count[state] == count for state in station_states):
                heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    # Process the heap
    while states_needed and heap:
        _, _, station, station_states = heapq.heappop(heap)
        if station in final_stations:
            continue
        final_stations.add(station)
        states_needed -= station_states
        for st in station_states:
            if st in Count:
                del Count[st]
        del stations[station]

        # Rebuild the heap with remaining stations
        heap = []
        for count in range(1, max_count + 1):
            for station, station_states in stations.items():
                if any(state in Count and Count[state] == count for state in station_states):
                    heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    return final_stations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
greedy_result, greedy_time = measure_runtime(greedy_set_covering, states_needed.copy(), stations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, states_needed.copy(), stations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, states_needed.copy(), stations.copy())

# Print results
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")

Greedy Algorithm: {'kfive', 'kthree', 'kone', 'ktwo'}, Time: 1.4332999995758655e-05
Heap-Based Algorithm: {'kfive', 'ktwo', 'kfour', 'kthree'}, Time: 7.308799999350413e-05
ILP Algorithm: ['ktwo', 'kthree', 'kfour', 'kfive'], Time: 0.011743835999993735
Greedy Algorithm Time: 1.4332999995758655e-05
Heap-Based Algorithm Time: 7.308799999350413e-05
ILP Algorithm Time: 0.011743835999993735


In [ ]:
states_needed = {"s1", "s2", "s3", "s4", "s5", "s6"}

stations = {
    "A": {"s1", "s2", "s3"},
    "B": {"s1", "s4"},
    "C": {"s2", "s5"},
    "D": {"s3", "s6"},
    "E": {"s4", "s5", "s6"}
}

# Greedy Algorithm
def greedy_set_covering(states_needed, stations):
    final_stations = set()
    while states_needed:
        best_station = None
        states_covered = set()
        for station, station_states in stations.items():
            covered = states_needed & station_states
            if len(covered) > len(states_covered):
                best_station = station
                states_covered = covered
        if best_station:
            final_stations.add(best_station)
            states_needed -= states_covered
            del stations[best_station]
        else:
            break
    return final_stations

# ILP Algorithm
def ilp_set_covering(states_needed, stations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    station_vars = pulp.LpVariable.dicts("Station", stations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([station_vars[station] for station in stations.keys()])
    for state in states_needed:
        prob += pulp.lpSum([station_vars[station] for station in stations.keys() if state in stations[station]]) >= 1
    prob.solve()
    selected_stations = [station for station in stations.keys() if station_vars[station].varValue == 1]
    return selected_stations

# Heap-Based Algorithm
def heap_set_covering(states_needed, stations):
    final_stations = set()
    Count = {state: 0 for state in states_needed}

    # Count occurrences of each state in the stations
    for station, station_states in stations.items():
        for state in station_states:
            if state in Count:
                Count[state] += 1

    heap = []

    # Function to calculate the number of states in common with states_needed
    def states_in_common(station_states):
        return len(states_needed & station_states)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for station, station_states in stations.items():
            if any(Count[state] == count for state in station_states):
                heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    # Process the heap
    while states_needed and heap:
        _, _, station, station_states = heapq.heappop(heap)
        if station in final_stations:
            continue
        final_stations.add(station)
        states_needed -= station_states
        for st in station_states:
            if st in Count:
                del Count[st]
        del stations[station]

        # Rebuild the heap with remaining stations
        heap = []
        for count in range(1, max_count + 1):
            for station, station_states in stations.items():
                if any(state in Count and Count[state] == count for state in station_states):
                    heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    return final_stations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
greedy_result, greedy_time = measure_runtime(greedy_set_covering, states_needed.copy(), stations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, states_needed.copy(), stations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, states_needed.copy(), stations.copy())

# Print results
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")

Greedy Algorithm: {'E', 'A'}, Time: 1.1156000027767732e-05
Heap-Based Algorithm: {'E', 'A'}, Time: 4.9620000027061906e-05
ILP Algorithm: ['A', 'E'], Time: 0.009903937000217411
Greedy Algorithm Time: 1.1156000027767732e-05
Heap-Based Algorithm Time: 4.9620000027061906e-05
ILP Algorithm Time: 0.009903937000217411
